In [1]:
import keras
from keras.preprocessing.image import load_img, img_to_array  ,array_to_img

In [2]:
from glob import glob

In [3]:
path= glob(r"C:\Users\guptaek\Deep learning\catvsdog_classifier\train"+"\dog*")

In [4]:
path_cat=glob(r"C:\Users\guptaek\Deep learning\catvsdog_classifier\train"+"\cat*")

In [5]:
import numpy as np

In [6]:
dogs=[]
cats=[]
for img in path[:1000]:
    dogs.append(img_to_array(load_img(img,target_size= (224,224)))) 
for img in path_cat[:1000]:
    cats.append(img_to_array(load_img(img,target_size= (224,224))))

In [7]:
Dogs=np.array(dogs)
Cats=np.array(cats)

In [8]:
Dogs.shape

(1000, 224, 224, 3)

In [9]:
Cats.shape

(1000, 224, 224, 3)

In [10]:
dog_labels=np.zeros(shape=[1000,1])
cat_labels=np.ones(shape=[1000,1])

In [11]:
X=np.concatenate((Dogs,Cats),axis=0)
Y=np.concatenate((dog_labels,cat_labels),axis=0)

In [12]:
print(X.shape , Y.shape)

(2000, 224, 224, 3) (2000, 1)


In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test , y_train , y_test= train_test_split(X,Y, test_size=0.2 , random_state=2)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1600, 224, 224, 3) (400, 224, 224, 3) (1600, 1) (400, 1)


In [14]:
from keras.applications import VGG16
from keras.layers import Flatten , Dense

In [29]:
from keras.applications.vgg16 import VGG16
model=keras.models.Sequential()
vgg=VGG16(input_shape=(224,224,3) , include_top=False)
vgg.trainable=False
#for layer in model.layers[:10]:
#    layer.trainable=False
model.add(vgg)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [30]:
model.compile(loss='binary_crossentropy' , optimizer='adam' , metrics=['accuracy'])

In [31]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [32]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25089     
Total params: 14,739,777
Trainable params: 25,089
Non-trainable params: 14,714,688
_________________________________________________________________


In [33]:
for i,layer in enumerate(model.layers[:-1]):
    print(i,layer.name, "-",layer.trainable)

0 vgg16 - False
1 flatten_2 - True


In [35]:
model.fit(x_train, y_train, batch_size=32, epochs=3)

Epoch 1/3
50/50 [==============================] - 516s 10s/step - loss: 1.6467 - accuracy: 0.9212
Epoch 2/3
50/50 [==============================] - 519s 10s/step - loss: 0.4497 - accuracy: 0.9769
Epoch 3/3
50/50 [==============================] - 518s 10s/step - loss: 0.0838 - accuracy: 0.9931


In [36]:
model.evaluate(x_test,y_test)

13/13 [==============================] - 131s 10s/step - loss: 1.6007 - accuracy: 0.9525


[1.6006577014923096, 0.9524999856948853]

In [40]:
test_dogs=[]
for img in path[1000:2000]:
    test_dogs.append(img_to_array(load_img(img,target_size= (224,224))))

In [42]:
Test_dogs=np.array(test_dogs)

In [43]:
Test_dogs.shape

(1000, 224, 224, 3)

In [65]:
Test_dogs_pred=[1.0 if i>0.5 else 0.0 for i in model.predict(Test_dogs) ]

In [48]:
from sklearn.metrics import confusion_matrix

In [56]:
Test_dogs_pred.shape

(1000, 1)

In [63]:
actual=dog_labels.reshape(-1,)

In [67]:
confusion_matrix(Test_dogs_pred, dog_labels, labels=[1,0])

array([[  0,  18],
       [  0, 982]], dtype=int64)

### using keras tuner

In [70]:
from kerastuner.tuners import RandomSearch

In [82]:
from keras.layers import Conv2D , MaxPool2D

In [96]:


def model_builder(hp):
    model=keras.Sequential()
    model.add(Conv2D( 32, kernel_size=(3,3), input_shape= (224,224,3) , activation='relu'))
    for i in range(hp.Int('num_layers',10,32, 2)):
        model.add(Conv2D( hp.Int('num_layers',32, 96, 16),kernel_size=(3,3),activation='relu'  ))
    model.add(MaxPool2D( 3,3 ))
    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'] )
    return model



In [97]:
tuner=RandomSearch(model_builder,
                   objective='val_accuracy',
                  executions_per_trial= 1,
                  max_trials=3,
                   directory='parc1',
                   project_name='TransferLearning prac'
                  )

INFO:tensorflow:Reloading Oracle from existing project parc1\TransferLearning prac\oracle.json


In [106]:
from keras.callbacks import EarlyStopping
es=EarlyStopping(monitor='accuracy' , mode='max' , patience= 2 , baseline= 0.7 )

In [107]:
tuner.search(x_train, y_train, epochs=3, batch_size= 32, callbacks=[es])


Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
num_layers        |30                |?                 

Epoch 1/3
50/50 [==============================] - 2583s 51s/step - loss: 0.7848 - accuracy: 0.5232
Epoch 2/3
50/50 [==============================] - 2228s 45s/step - loss: 0.6931 - accuracy: 0.5078


ValueError: Objective value missing in metrics reported to the Oracle, expected: ['val_accuracy'], found: dict_keys(['loss', 'accuracy'])

In [ ]:
tuner.evauate(x_test,y_test)

## using Resnet50 architecture

In [ ]:
from keras.applications import ResNet50


In [ ]:
resnet = ResNet50(input_shape=(224,224,3), include_top=False)
for layer in resnet.layers:
    layer.trainable=False
model=keras.models.Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))


In [ ]:
resnet.summary()

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy' , optimizer='adam' , metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=4, batch_size=50)

In [ ]:
model.evaluate(x_test, y_test)

In [19]:
li = [2, 1, 4, 6, 2, 4, 3]

In [31]:
k=0
m=[]
for i in li:
    if(i%2 == 0 and (i not in m)):
        m.append(i)
        print(k)
    k+=1

0
2
3


In [28]:
m

[2, 4, 6]

In [ ]:
for k in range(len(li)):
    li[k]

In [32]:
li.remove(2)

In [33]:
li

[1, 4, 6, 2, 4, 3]